# Adding interactivity
In the preceding notebook, we saw how the size and color of our data allowed use to convey much more information in our plot. We can add still more information to our plot by adding **interactivity**. 

In this notebook, we'll add a slider to our notebook, allowing the user to move across years while updating our plot to display data for that year. In this case, we'll use [iPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), a set of nifty little controllers we can embed in our notebooks. We haven't time to into much detail with these, but this notebook serves as a "proof of concept" as well a chance to discuss the chain of events that allows this all to happen.   

This notebook builds of the example from the last...

### Step 1. Import the usual Bokeh objects

In [1]:
#Import io objects
from bokeh.io import output_notebook, show

#Import the figure object
from bokeh.plotting import figure

#Import models
from bokeh.models import (ColumnDataSource, 
                          HoverTool, 
                          LinearInterpolator, 
                          CategoricalColorMapper, 
                          NumeralTickFormatter)   #Allows us to show axes values in specific formats

#Import all color palettes
from bokeh.palettes import *

from bokeh.models import NumeralTickFormatter #import Bokeh's number formatter, for x-axis

#Activate Bokeh, sending figures to the notebook
output_notebook()

Loading BokehJS ...

### Step 2. Read the data into a Bokeh ColumnDataSource object

In [2]:
#import the data as a dataframe then as a ColumnDataSource object
import pandas as pd
data = pd.read_csv('./data/gapminder.csv',thousands=',',index_col='Year')
theCDS = ColumnDataSource(data.loc[2010])

### Step 3. Construct the basic plot, as before

In [3]:
#Create a styling dictionary for our plot
PLOT_OPS = {'title':'2010',
            'height':500,
            'width':900,
            'x_axis_type':'log',
            'x_axis_label':'Log(Income)',
            'y_axis_label':'Life expectancy (years)',
            'x_range':(100,100000),
            'y_range':(0,100),
            'toolbar_location':"below"
           }

In [4]:
#Create the population size mapper
size_mapper = LinearInterpolator(x=[data.loc[2010]['population'].min(),
                                    data.loc[2010]['population'].max()],
                                 y=[5,50])

In [5]:
#Create the region color mapper
color_mapper = CategoricalColorMapper(factors=data.loc[2010]['region'].unique(),
                                      palette=Spectral6)

In [6]:
#Plot again, now using the color_mapper we just created
hover = HoverTool(tooltips='@Country')
p = figure(**PLOT_OPS,)
p.circle(x='income',
         y='life',
         source=theCDS,
         size={'field':'population','transform':size_mapper},
         color={'field':'region','transform':color_mapper},
         alpha=0.6,
         legend='region'
        )
p.legend.border_line_color = 'red'
p.right = p.legend
#Set the x axis to be in dollars with commas
p.xaxis.formatter = NumeralTickFormatter(format='$0,')
#Add the hover tool
p.add_tools(hover)
show(p)

## Adding interativity - with iPyWidgets
* `ipywidgets` - enable HTML like controls in the notebook (the `interact` widget is a simple slider)
* `push_notebook` - enable Bohek to "write" back to the notebook

In [7]:
#Enable HTML like controls in the notebook
from ipywidgets import interact

In [8]:
#Allows Bokeh to write back to the notebool
from bokeh.io import push_notebook

Now is when things take a turn. 

First we need to define a Python function to handle a **callback**. We call this function `update` and it has one parameter, `year`, which is the year of data we want to visualize. This function constructs a new ColumnDataSource from the original `data` dataframe, and then updates the data in the original ColumnDataSource object (i.e., the one used in the plot). It also updates the title of the plot, and then "pushes" the changes to the existing plot. 

This `update` function does not run until called. Instead, the code block first constructs and shows the plot as above, but with one key exception: the **show** command has an extra parameter: `notebook_handle=True`. This enables other code cells to interact with the plot generated. 

After running this code cell, it should all look the same as before, BUT our plot is now listening for the `update` function to be called and when it does, the `theCDS` will have new data, and our plot will change. 

In [9]:
#Function to revise the data in the ColumnDataSource object
def update(year):
    new_data = ColumnDataSource(data.loc[year]).data #Select data from a different year
    theCDS.data = new_data #Update the data in the ColumnDataSource to the revised data
    p.title.text=str(year) #Update the title of our plot
    push_notebook()        #Push out a revised figure
    

#Plot again, now using the color_mapper we just created
hover = HoverTool(tooltips='@Country')
p = figure(**PLOT_OPS)
p.circle(x='income',
         y='life',
         source=theCDS,
         size={'field':'population','transform':size_mapper},
         color={'field':'region','transform':color_mapper},
         alpha=0.6,
         legend='region'
        )
p.legend.border_line_color = 'red'
p.right = p.legend
p.xaxis.formatter = NumeralTickFormatter(format='$0,')
p.add_tools(hover)
show(p,notebook_handle=True);

We can update our script using code, just by running the `update` function, supplying the year we want to view

In [10]:
#Update the graph to show data for a different year
update(1955)

Or, we can add an interface for our interactivity. Here we add a slider that runs the `update` function with whatever year is selected in the slider. 

In [11]:
#Add a simple iPyWidget slider that runs the `update` script when changed
interact(update,year=(1950,2015,1));

interactive(children=(IntSlider(value=1982, description='year', max=2015, min=1950), Output()), _dom_classes=(…